In [100]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, make_scorer
from sklearn.model_selection import GridSearchCV
from itertools import product

In [14]:
def evaluate_model(model_type, X_train, X_test, y_train, y_test):
    """Train a model and return evaluation metrics as a dict (no printing)."""
    if model_type == 'svm':
        model = SVC(kernel='rbf', C=0.1, random_state=42, probability=True)
    elif model_type == 'rf':
        model = RandomForestClassifier(max_depth=5, min_samples_split=5, random_state=42)
    elif model_type == 'logistic':
        model = LogisticRegression(penalty='l2', C=0.1, random_state=42, max_iter=1000)
    elif model_type == 'gradient_boosting':
        model = GradientBoostingClassifier(max_depth=3, random_state=42)
    elif model_type == 'knn':
        model = KNeighborsClassifier(n_neighbors=10)
    elif model_type == 'decision_tree':
        model = DecisionTreeClassifier(max_depth=5, random_state=42)
    else:
        raise ValueError(f"Unsupported model type: {model_type}")

    model.fit(X_train, y_train)

    # Train/test predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    metrics = {
        "train_accuracy": accuracy_score(y_train, y_train_pred),
        "train_precision": precision_score(y_train, y_train_pred),
        "train_recall": recall_score(y_train, y_train_pred),
        "train_f1": f1_score(y_train, y_train_pred),
        "test_accuracy": accuracy_score(y_test, y_test_pred),
        "test_precision": precision_score(y_test, y_test_pred),
        "test_recall": recall_score(y_test, y_test_pred),
        "test_f1": f1_score(y_test, y_test_pred)
    }

    return metrics


In [15]:
def build_clustering_dataset_optimized(
    data,
    lepol_time,
    kiln_time,
    refroidisseur_time,
    type="mean",
    tol=0.01,
):
    """
    Optimized clustering dataset builder (fixed indexing & labelling).
    - Rows are indexed by window end time (measurement time).
    - A window is kept only if every minute inside it exists in the raw data.
    - Input features taken at window start.
    - Lepol/kiln features taken only from their respective windows.
    - Labels: compare quality(end_time) with quality(end_time - 1 minute).
      If prev minute exists and abs(diff) > tol and neither is 0/NaN -> label.
    """
    data = data.copy()
    data.index = pd.to_datetime(data.index)
    # keep first row when timestamps duplicate
    data = data[~data.index.duplicated(keep='first')]

    total_window = lepol_time + kiln_time + refroidisseur_time

    lepol_features = [
        'Température sur chambre chaude ',
        'Temperature Air Secondaire',
        'Oxygene Sortie Four',
        'NOx Sortie Four',
        'SO2 Sortie Four',
        'CO Sortie Four'
    ]

    kiln_features = [
        'Intensite Moteur Four',
        'Temperature Zone'
    ]

    input_features = [
        'Debit moyen entree farine Four  (CF)',
        'Debit Injection SQ7',
        'Debit moyen SOLVEN in Kiln'
    ]

    # build full minute index and reindex (so integer positions correspond to minutes)
    min_time = data.index.min()
    max_time = data.index.max()
    full_range = pd.date_range(start=min_time, end=max_time, freq='min')
    data_reindexed = data.reindex(full_range)

    # mask of minutes that actually exist in original data (True if present)
    valid_mask = data_reindexed.index.isin(data.index)

    # arrays for fast access
    quality_arr = data_reindexed['Mesure traitee CaOl Alcatron'].values
    input_arrays = {col: data_reindexed[col].values for col in input_features}
    lepol_arrays = {col: data_reindexed[col].values for col in lepol_features}
    kiln_arrays = {col: data_reindexed[col].values for col in kiln_features}

    # compute valid start indices (integer minute offsets in full_range)
    max_start_idx = len(data_reindexed) - total_window - 1
    if max_start_idx < 0:
        return pd.DataFrame()

    valid_starts = []
    for i in range(max_start_idx + 1):
        lepol_end = i + lepol_time
        kiln_end = lepol_end + kiln_time
        refroid_idx = i + total_window

        # require that every minute used by the window exists in original data
        if (refroid_idx < len(valid_mask) and
            valid_mask[i:lepol_end + 1].all() and
            valid_mask[lepol_end:kiln_end + 1].all() and
            valid_mask[refroid_idx]):
            valid_starts.append(i)

    if not valid_starts:
        return pd.DataFrame()

    valid_starts = np.array(valid_starts)
    n_windows = len(valid_starts)
    end_indices = valid_starts + total_window
    end_times = full_range[end_indices]

    print(f"Building {n_windows} valid windows...")

    # build result dictionary using end_times as Time (measurement timestamp)
    result_dict = {
        'Time': end_times,
        'Mesure traitee CaOl Alcatron': quality_arr[end_indices]
    }

    # input features at start
    for col in input_features:
        result_dict[col] = input_arrays[col][valid_starts]

    # lepol / kiln features
    if type == "mean":
        for col in lepol_features:
            arr = lepol_arrays[col]
            means = np.empty(n_windows, dtype=float)
            for j, start_idx in enumerate(valid_starts):
                means[j] = np.nanmean(arr[start_idx:start_idx + lepol_time + 1])
            result_dict[f"{col}_lepol_mean"] = means

        for col in kiln_features:
            arr = kiln_arrays[col]
            means = np.empty(n_windows, dtype=float)
            for j, start_idx in enumerate(valid_starts):
                kstart = start_idx + lepol_time
                means[j] = np.nanmean(arr[kstart:kstart + kiln_time + 1])
            result_dict[f"{col}_kiln_mean"] = means

    elif type == "segment":
        for col in lepol_features:
            arr = lepol_arrays[col]
            segments = []
            for j, start_idx in enumerate(valid_starts):
                segments.append(arr[start_idx:start_idx + lepol_time + 1])
            result_dict[f"{col}_lepol_segment"] = segments

        for col in kiln_features:
            arr = kiln_arrays[col]
            segments = []
            for j, start_idx in enumerate(valid_starts):
                kstart = start_idx + lepol_time
                segments.append(arr[kstart:kstart + kiln_time + 1])
            result_dict[f"{col}_kiln_segment"] = segments

    # build dataframe (index = end_time)
    clustering_data = pd.DataFrame(result_dict)
    clustering_data.set_index("Time", inplace=True)

    # ------------------------
    # Vectorized, correct labelling
    # ------------------------
    # default
    labels_arr = np.full(n_windows, -1, dtype=int)

    # current and previous indices in the reindexed full_range
    curr_idx = end_indices              # integer positions in full_range
    prev_idx = curr_idx - 1

    # prev must be in-bounds and correspond to an actual original timestamp
    valid_prev = (prev_idx >= 0) & valid_mask[prev_idx]

    if valid_prev.any():
        curr_vals = quality_arr[curr_idx[valid_prev]]
        prev_vals = quality_arr[prev_idx[valid_prev]]

        # require not NaN and not zero (as before)
        ok_vals = (~np.isnan(curr_vals)) & (~np.isnan(prev_vals)) & (curr_vals != 0) & (prev_vals != 0)

        # label where difference exceeds tolerance
        label_mask = ok_vals & (np.abs(curr_vals - prev_vals) > tol)
        labels = np.where(curr_vals[label_mask] <= 3.5, 1, 0)

        # place labels into labels_arr at positions corresponding to valid_prev True
        positions = np.flatnonzero(valid_prev)      # positions into the windows array
        label_positions = positions[label_mask]    # window indices to label
        labels_arr[label_positions] = labels

    # attach labels to dataframe
    clustering_data['Labels'] = labels_arr

    return clustering_data


In [16]:
csv_path = "/Users/foucauld/Desktop/stage_centre_Borelli/Clean_code/clean_raw_data.csv"

data = pd.read_csv(csv_path, sep=',')
data['Time'] = pd.to_datetime(data['Time'])
data.set_index('Time', inplace=True)
print(len(data))
# Keep the first row for each timestamp
data = data[~data.index.duplicated(keep='first')]
print(len(data))

96358
94919


In [17]:
def param_grid_search(data, lepol_grid, kiln_grid, refroidisseur_grid, model_types):
    results = []

    for lepol_time, kiln_time, refroidisseur_time in product(lepol_grid, kiln_grid, refroidisseur_grid):
        print(f"\n--- Params: lepol={lepol_time}, kiln={kiln_time}, refroidisseur={refroidisseur_time} ---")

        try:
            # Build dataset
            clustering_data = build_clustering_dataset_optimized(
                data, lepol_time, kiln_time, refroidisseur_time,
                type="mean", tol=0.01
            )

            # Balance classes
            count_0 = (clustering_data['Labels'] == 0).sum()
            count_1 = (clustering_data['Labels'] == 1).sum()
            if count_0 == 0 or count_1 == 0:
                print("Skipping: one class has 0 samples")
                continue

            class_0 = clustering_data[clustering_data['Labels'] == 0]
            class_1 = clustering_data[clustering_data['Labels'] == 1]
            class_1_downsampled = class_1.sample(count_0, random_state=42)
            balanced_data = pd.concat([class_0, class_1_downsampled])

            X = balanced_data.drop(columns=['Labels', 'Mesure traitee CaOl Alcatron'])
            y = balanced_data['Labels']

            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X)

            X_train, X_test, y_train, y_test = train_test_split(
                X_scaled, y, test_size=0.2, random_state=42, stratify=y, shuffle=True
            )

            # Loop over models
            for model_type in model_types:
                metrics = evaluate_model(model_type, X_train, X_test, y_train, y_test)
                results.append({
                    "model": model_type,
                    "lepol_time": lepol_time,
                    "kiln_time": kiln_time,
                    "refroidisseur_time": refroidisseur_time,
                    **metrics
                })

        except Exception as e:
            print(f"Error with params ({lepol_time}, {kiln_time}, {refroidisseur_time}): {e}")

    return pd.DataFrame(results)


In [18]:
param_grid = {
    "lepol": [15, 20, 25, 30, 35],
    "kiln": [30, 35, 40, 45, 50, 55, 60, 65, 70],
    "refroidisseur": [25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80]
}

model_types = ["rf", "svm", "gradient_boosting", "knn", "decision_tree"]

results_df = param_grid_search(
    data,
    param_grid["lepol"],
    param_grid["kiln"],
    param_grid["refroidisseur"],
    model_types
)

# Sort by test F1 (best metric for balanced performance)
print(results_df.sort_values(by="test_f1", ascending=False).head(10))



--- Params: lepol=15, kiln=30, refroidisseur=25 ---
Building 94709 valid windows...

--- Params: lepol=15, kiln=30, refroidisseur=30 ---
Building 94694 valid windows...

--- Params: lepol=15, kiln=30, refroidisseur=35 ---
Building 94679 valid windows...

--- Params: lepol=15, kiln=30, refroidisseur=40 ---
Building 94664 valid windows...

--- Params: lepol=15, kiln=30, refroidisseur=45 ---
Building 94649 valid windows...

--- Params: lepol=15, kiln=30, refroidisseur=50 ---
Building 94634 valid windows...

--- Params: lepol=15, kiln=30, refroidisseur=55 ---
Building 94619 valid windows...

--- Params: lepol=15, kiln=30, refroidisseur=60 ---
Building 94604 valid windows...

--- Params: lepol=15, kiln=30, refroidisseur=65 ---
Building 94599 valid windows...

--- Params: lepol=15, kiln=30, refroidisseur=70 ---
Building 94594 valid windows...

--- Params: lepol=15, kiln=30, refroidisseur=75 ---
Building 94589 valid windows...

--- Params: lepol=15, kiln=30, refroidisseur=80 ---
Building 945

In [114]:
results_df.sort_values(by="test_f1", ascending=False).to_csv("grid_search_results.csv", index=False)

In [96]:
grid_search_results = pd.read_csv("grid_search_results.csv")
grid_search_results.sort_values(by="test_f1", ignore_index=True)
print(grid_search_results.head())

               model  lepol_time  kiln_time  refroidisseur_time  \
0  gradient_boosting          20         30                  40   
1  gradient_boosting          35         30                  25   
2  gradient_boosting          25         35                  25   
3  gradient_boosting          30         35                  25   
4  gradient_boosting          25         30                  30   

   train_accuracy  train_precision  train_recall  train_f1  test_accuracy  \
0        0.886002         0.924494      0.840871  0.880702       0.826087   
1        0.899413         0.923623      0.871022  0.896552       0.829431   
2        0.887866         0.915771      0.854515  0.884083       0.822742   
3        0.896899         0.934066      0.854271  0.892388       0.819398   
4        0.890377         0.916221      0.859532  0.886972       0.822742   

   test_precision  test_recall   test_f1  
0        0.829932     0.818792  0.824324  
1        0.850000     0.798658  0.823529  
2    

In [101]:
grid_search_results["Total_time"] = grid_search_results[["lepol_time", "kiln_time", "refroidisseur_time"]].sum(axis=1)
unique_time = grid_search_results["Total_time"].unique()
df_time_analysis = pd.DataFrame()
df_time_analysis["Time"] = np.sort(unique_time)
df_time_analysis.set_index("Time", inplace=True)

for time in unique_time:
    subset = grid_search_results[grid_search_results["Total_time"] == time]

    # Average index and scores
    df_time_analysis.loc[df_time_analysis.index == time, "average_index"] = subset.index.to_numpy().mean()
    df_time_analysis.loc[df_time_analysis.index == time, "average_test_f1"] = subset["test_f1"].mean()

    # Best test F1 score
    df_time_analysis.loc[df_time_analysis.index == time, "best_test_f1"] = subset["test_f1"].max()

    # Best train F1 score corresponding to the best test F1 score
    best_idx = subset["test_f1"].idxmax()
    df_time_analysis.loc[df_time_analysis.index == time, "best_train_f1"] = subset.loc[best_idx, "train_f1"]

    # Best model and times
    df_time_analysis.loc[df_time_analysis.index == time, "best_model"] = subset.loc[best_idx, "model"]
    df_time_analysis.loc[df_time_analysis.index == time, "best_lepol_time"] = subset.loc[best_idx, "lepol_time"]
    df_time_analysis.loc[df_time_analysis.index == time, "best_kiln_time"] = subset.loc[best_idx, "kiln_time"]
    df_time_analysis.loc[df_time_analysis.index == time, "best_refroidisseur_time"] = subset.loc[best_idx, "refroidisseur_time"]

print(df_time_analysis)

      average_index  average_test_f1  best_test_f1  best_train_f1  \
Time                                                                
70       724.000000         0.733974      0.790698       0.729218   
75       497.933333         0.758031      0.790036       0.817360   
80       602.766667         0.755438      0.807018       0.816952   
85       540.740000         0.756983      0.815331       0.884083   
90       378.506667         0.767669      0.824324       0.880702   
95       726.750000         0.742622      0.791519       0.883478   
100     1275.032000         0.720837      0.797203       0.899130   
105     1449.220000         0.713051      0.780488       0.894602   
110     1230.274286         0.725261      0.801418       0.891587   
115     1159.117949         0.726594      0.798611       0.885590   
120     1643.880952         0.705761      0.763251       0.890612   
125     1930.131818         0.696483      0.757895       0.907519   
130     1732.695455         0.7034

In [102]:
clustering_data_optimal_time = build_clustering_dataset_optimized(data, lepol_time=20, kiln_time=30, refroidisseur_time=40, type="mean",tol=0.01,)

Building 94649 valid windows...


In [106]:
count_0 = (clustering_data_optimal_time['Labels'] == 0).sum()
count_1 = (clustering_data_optimal_time['Labels'] == 1).sum()
if count_0 == 0 or count_1 == 0:
    print("Skipping: one class has 0 samples")

class_0 = clustering_data_optimal_time[clustering_data_optimal_time['Labels'] == 0]
class_1 = clustering_data_optimal_time[clustering_data_optimal_time['Labels'] == 1]
class_1_downsampled = class_1.sample(count_0, random_state=42)
balanced_data = pd.concat([class_0, class_1_downsampled])

X = balanced_data.drop(columns=['Labels', 'Mesure traitee CaOl Alcatron'])
y = balanced_data['Labels']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y, shuffle=True
)

In [112]:
gb = GradientBoostingClassifier(random_state=42)

param_grid = {
    'n_estimators': [200, 300, 400, 500],          # extend around 300
    'learning_rate': [0.01, 0.05, 0.1, 0.2],       # finer grid, try slightly higher
    'max_depth': [3, 4, 5],                        # focus near 4
    'min_samples_split': [2, 5, 10, 20],           # allow more conservative splits
    'min_samples_leaf': [1, 3, 5, 10],             # test bigger leaves for stability
    'subsample': [0.7, 0.8, 0.9, 1.0],             # stochastic boosting can help
    'max_features': ['sqrt', 'log2', None]         # try feature subsampling
}



grid_search = GridSearchCV(
    estimator=gb,
    param_grid=param_grid,
    scoring='f1',   
    cv=5,           
    n_jobs=-1,     
    verbose=0
)

grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best cross-validated F1:", grid_search.best_score_)

# Evaluate on test set
best_gb = grid_search.best_estimator_
y_pred = best_gb.predict(X_test)
print("Test F1 Score:", f1_score(y_test, y_pred))

Best parameters: {'learning_rate': 0.1, 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 500, 'subsample': 0.9}
Best cross-validated F1: 0.7915023606158725
Test F1 Score: 0.8191126279863481
